Import libraries

In [ ]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib nbagg
%matplotlib inline
import IPython.display as ipd
from IPython.display import Audio
import os, sys
from scipy.io import wavfile
from sklearn.preprocessing import LabelEncoder, scale
import time
import tensorflow as tf
tf.config.list_physical_devices('GPU')
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
from numpy import asarray
from numpy import savetxt
from tensorflow.keras.callbacks import ReduceLROnPlateau

import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.utils import to_categorical
from keras.utils import np_utils
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, Input, MaxPooling2D, Conv1D, MaxPooling1D, Lambda
from tensorflow.keras import losses
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/My Drive/Data/Audio Files/"
labels = os.listdir(path)
print(labels)

In [ ]:
ignored_files = {"_background_noise_", "LICENSE", "README.md", "testing_list.txt", "validation_list.txt", "Extracted Arrays",".ipynb_checkpoints"}
labels = [x for x in os.listdir(path) if x not in ignored_files]
print(labels)
print("Total labels: ", len(labels))

In [ ]:
time_start = time.time()

number_of_recordings = []

for label in labels:
  waves = [f for f in os.listdir(path + '/'+ label) if f.endswith('.wav')]
  number_of_recordings.append(len(waves))

plt.figure(figsize=(30,5))
index = np.arange(len(labels))
plt.bar(index, number_of_recordings)
plt.xlabel('Commands', fontsize=12)
plt.ylabel('No of recordings', fontsize=12)
plt.xticks(index, labels, fontsize=15, rotation=60)
plt.title('No. of recordings for each command')
plt.show()

print('Run time: {} mins'.format((time.time()-time_start)/60))

In [ ]:

time_start = time.time()

duration_of_recordings=[]
for label in labels:
    print(label)
    waves = [f for f in os.listdir(path + '/'+ label) if f.endswith('.wav')]
    for wav in waves:
        sample_rate, samples = wavfile.read(path + '/' + label + '/' + wav)
        duration_of_recordings.append(float(len(samples)/sample_rate))
    
plt.hist(np.array(duration_of_recordings))

print('Run time: {} mins'.format((time.time()-time_start)/60))

In [ ]:
savetxt('number_of_recordings.csv', number_of_recordings, delimiter=',')
savetxt('duration_of_recordings.csv', duration_of_recordings, delimiter=',')

In [ ]:
#!cp number_of_recordings.csv "/content/gdrive/My Drive/Automatic Speech Recognition/Data/Google AI: Speech Commands Dataset/"
#!cp duration_of_recordings.csv "/content/gdrive/My Drive/Automatic Speech Recognition/Data/Google AI: Speech Commands Dataset/"

In [ ]:
sum(number_of_recordings)

In [ ]:
path = "/content/gdrive/My Drive/Automatic Speech Recognition/Data/Google AI: Speech Commands Dataset/Extracted Data/"
path2 = "/content/gdrive/My Drive/Automatic Speech Recognition/Data/Google AI: Speech Commands Dataset/Extracted Data/Extracted Arrays/"

In [ ]:
time_start = time.time()

path = "/content/drive/My Drive/Data/Audio Files/"
path2 = "/content/drive/My Drive/concatenated_audio/"

for label in labels:

    audio_onesec = []
    label_list = []
    

    print(label)
    waves = [f for f in os.listdir(path + '/'+ label) if f.endswith('.wav')]

    for wav in waves:
        samples, sample_rate = librosa.load(path + '/' + label + '/' + wav, sr = 16000)
        samples = librosa.resample(samples, 16000, 8000)
        if(len(samples) == 8000) : 
            audio_onesec.append(samples)
            label_list.append(label)


    np.save(os.path.join(path2, str(label) + '_audio_onesec'), audio_onesec)
    np.save(os.path.join(path2, str(label) + '_label_list'), label_list)
    print(label, 'is done')

print('Run time: {}'.format(time.time()-time_start))

In [ ]:
audio_samples1 = np.load(os.path.join(path2, 'bed_audio_onesec.npy'))
audio_samples1.shape

In [ ]:
labels1 = np.load(os.path.join(path2, 'bed_label_list.npy'))
labels1.shape

In [ ]:
#concatenat whole datase and labels as well suvh that mapping
all_audio = np.empty([0, 8000])

for label in labels:
  audiosamples = np.load(os.path.join(path2, str(label) + '_audio_onesec.npy'))
  #print(audiosamples.shape)
  all_audio = np.concatenate((all_audio, audiosamples), axis=0)
  print(all_audio.shape)

print('done')
  
np.save(os.path.join(path2, 'all_audio'), all_audio)

In [ ]:
all_labels = np.empty([0,])

for label in labels:
  current_labels = np.load(os.path.join(path2, str(label) + '_label_list.npy'))
  all_labels = np.concatenate((all_labels, current_labels), axis=0)
  print(all_labels.shape)

print('done')
  
np.save(os.path.join(path2, 'all_labels'), all_labels)

In [ ]:
all_audio = np.load(os.path.join(path2, 'all_audio.npy'))
all_labels = np.load(os.path.join(path2, 'all_labels.npy'))

In [ ]:
all_labels = np.reshape(all_labels, (58252, 1))
print(all_labels.shape)

In [ ]:
all_labels = np.load(os.path.join(path2, 'all_labels.npy'))
all_labels_reshaped = np.reshape(all_labels, (58252 , 1))
print(all_labels_reshaped.shape)
np.save(os.path.join(path2, 'all_labels'), all_labels_reshaped)
print('all_labels saved')

In [ ]:
#one hot encoding
all_labels = np.load(os.path.join(path2, 'all_labels.npy'))

le = LabelEncoder()
y=le.fit_transform(all_labels)
classes= list(le.classes_)

y = np_utils.to_categorical(y, num_classes=len(labels))

print(y.shape)

np.save(os.path.join(path2, 'all_labels_encoded'), y)

In [ ]:
path = "/content/drive/My Drive/Data/Audio Files/"
path2 = "/content/drive/My Drive/concatenated_audio/"

In [ ]:
y = np.load(os.path.join(path2, 'all_labels_encoded.npy'))
audio = np.load(os.path.join(path2, 'all_audio.npy'))

In [ ]:
y.shape

In [ ]:
audio = np.expand_dims(audio, axis=2)
audio.shape

In [ ]:
x_train, x_test, Y_train, Y_test = train_test_split(np.array(audio), np.array(y), stratify=y, test_size = 0.3, random_state=32, shuffle=True)

In [ ]:
del audio

In [ ]:
K.clear_session()

num_labels = len(labels)

model = Sequential()

model.add(Conv1D(filters=512, kernel_size=3, padding='valid', activation='relu', input_shape=(8000, 1)))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.2))

model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.2))

model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.2))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))                                
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.2))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.2))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.2))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.2))


model.add(Flatten())

model.add(Dense(256, activation='relu'))

model.add(Dense(num_labels, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
model.summary()

In [ ]:
batch_size = 128
num_epochs = 240

from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.callbacks import TensorBoard, CSVLogger
from keras.models import load_model
from numpy.testing import assert_allclose

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)  
path_checkpoint = "H:\whizkey\FINAL PAPER/weights_of_audio.best.hdf5"
mc = ModelCheckpoint(path_checkpoint, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, 
                              mode='auto',min_delta=0.0001, min_lr=0.0000001)

csv_logger = CSVLogger('H:\whizkey\FINAL PAPER/csv_logger_audio.csv', append=True, separator=',')

cbks = [reduce_lr, tensorboard, es, csv_logger, mc]

history = model.fit(x_train, Y_train, batch_size=batch_size, epochs=num_epochs,
                    callbacks = cbks, verbose=1, validation_data=(x_test, Y_test))

print('Run time: {} mins'.format((time.time()-time_start)/60))

In [ ]:
score = model.evaluate(x_train, Y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, Y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Below code is for Speech Recgonition: To be used by developer only (Windows OS), for end-user "Automatic_Speech_Recognition_End_User.ipynb"

Load Weights

In [ ]:
from keras.models import load_model
model = load_model('H:\whizkey\FINAL PAPER\weights_of_audio.best.hdf5')

In [ ]:
score = model.evaluate(x_train, Y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, Y_test, verbose=0)
print("Testing Accuracy: ", score[1])

In [ ]:
print(x_test.shape)
print(x_train.shape)

In [ ]:
labels = ['eight', 'sheila', 'nine', 'yes', 'one', 'no', 'left', 'tree', 'bed', 'bird', 'go', 'wow', 'seven', 'marvin', 'dog', 'three', 'two', 'house', 'down', 'six', 'five', 'off', 'right', 'cat', 'zero', 'four', 'stop', 'up', 'on', 'happy']

In [ ]:
all_labels = np.load(os.path.join("all_labels.npy"))

le = LabelEncoder()
y=le.fit_transform(all_labels)
classes= list(le.classes_)

y = np_utils.to_categorical(y, num_classes=len(labels))

print(y.shape)

#np.save(os.path.join(path2, 'all_labels_encoded'), y)

Predict probabilities for all speeches.
Maximum probabilty with a threshold of 0.5 is chosen as predicted speech.

In [ ]:
from datetime import datetime 
import pytz 
import csv
import random
  
# get the standard UTC time  
UTC = pytz.utc 
IST = pytz.timezone('Asia/Kolkata') 

def predict(audio,n):
    prob=model.predict_proba(audio.reshape(1,8000,1))
    index=np.argmax(prob[0])
    #print(prob[0])  #Print all probabilities
    #print(max(prob[0]))     #Print max probability
    if max(prob[0]) > 0.5:
        #print(index)
        f=open(r'H:\\whizkey\\FINAL PAPER\\Excel files\\Texter' + '{0}'.format(n)+'.txt', 'a')
        f.write(str(datetime.now(IST)) + "," + classes[index] + "\n")
        f.close()
        return classes[index]

Predictions on random speech commands from test set.

In [ ]:
# Dont run
import random
index=random.randint(0,len(x_test)-1)
print(index)
samples=x_test[index].ravel()
print("Audio:",classes[np.argmax(Y_test[index])])
ipd.Audio(samples, rate=8000)
print("Text:",predict(samples))

Dependencies for Silence Detection

In [ ]:
!pip install pydub &> /dev/null
!pip install sounddevice &> /dev/null
!pip install soundfile &> /dev/null
!sudo apt-get install libportaudio2 &> /dev/null
!pip install pysndfx &> /dev/null
!pip install python_speech_features &> /dev/null
#!sudo apt-get install sox &> /dev/null

In [ ]:
# Import the AudioSegment class for processing audio and the 
# split_on_silence function for separating out silent chunks.
from pydub import AudioSegment
from pydub.silence import split_on_silence

# Define a function to normalize a chunk to a target amplitude.
def match_target_amplitude(aChunk, target_dBFS):
    ''' Normalize given audio chunk '''
    change_in_dBFS = target_dBFS - aChunk.dBFS
    return aChunk.apply_gain(change_in_dBFS)

In [ ]:
from pathlib import Path
AudioSegment.converter = r"H:\\whizkey\\FINAL PAPER\\ffmpeg\\ffmpeg.exe"
AudioSegment.ffprobe   = r"H:\\whizkey\\FINAL PAPER\\ffmpeg\\ffprobe.exe"
 
print (AudioSegment.converter)
print (AudioSegment.ffprobe)
 
my_file = Path(r"H:\\whizkey\\FINAL PAPER\\cont speech\\newer10.mp3")
print ('ID1 : %s' % my_file) 
audio = AudioSegment.from_file_using_temporary_files(my_file)    

In [ ]:
from os import path
from pydub import AudioSegment

# files                                                                         
src = r"H:\\whizkey\\FINAL PAPER\\cont speech\\newer10.mp3"
dst = r"H:\\whizkey\\FINAL PAPER\\cont speech\\newer10.wav"

# convert wav to mp3                                                            
song1 = AudioSegment.from_file_using_temporary_files(src)
song1.export(dst, format="wav")

Calculate dBFS (Decibels relative to full scale)

In [ ]:
dBFS = song1.dBFS

In [ ]:
chunks = split_on_silence(
    song1,
    min_silence_len = 200,          #Set minimum silence length threshold to 100 ms
    silence_thresh = dBFS-16
)

Split the continuous audio into individual speech commands.

In [ ]:
# Process each chunk with your parameters
n = random. randint(0,9999999999999999999999) 

for i, chunk in enumerate(chunks):
    # Create a silence chunk that's 0.5 seconds (or 500 ms) long for padding.
    silence_chunk = AudioSegment.silent(duration=200)

    # Add the padding chunk to beginning and end of the entire chunk.
    audio_chunk = silence_chunk + chunk + silence_chunk

    # Normalize the entire chunk.
    normalized_chunk = match_target_amplitude(audio_chunk, -20.0)

    # Export the audio chunk with new bitrate.
    print("Exporting chunker{0}.wav.".format(i))
    k = normalized_chunk.export(
        r'H:\\whizkey\\FINAL PAPER\\Chunkers\\chunker{0}.wav' .format(i),
        bitrate = "192k",
        format = "wav"
    )

Resample each speech command to 8000 Hz

In [ ]:
time_start = time.time()
ctr = 0
for i, chunk in enumerate(chunks):
    ctr += 1
    melspec1 = []
    filepath=r'H:\\whizkey\\FINAL PAPER\\Chunkers\\chunker{0}.wav'.format(i)
    print(filepath)
    
    samples, sample_rate = librosa.load(filepath, sr = 16000)
    samples = librosa.resample(samples, 16000, 8000)
    samples.resize(8000)

    pred = predict(samples,n)         #Prediction after resampling to 8000Hz    
    print("Text:",pred)

print('Run time: {}'.format(time.time()-time_start))
print('Total speech commands: ',ctr)
dataframe1 = pd.read_csv(r"H:\\whizkey\\FINAL PAPER\\Excel files\\texter{0}.txt".format(n),header=None) 
dataframe1.to_csv(r"H:\\whizkey\\FINAL PAPER\\Excel files\\texter{0}.csv".format(n),index = None)
print("H:\\whizkey\\FINAL PAPER\\Excel files\\texter{0}.csv".format(n))

Play individual speech command

In [ ]:
ipd.Audio('/content/chunker38.wav', rate=8000)